In [1]:
#Import and initialize Google Earth Engine
import ee
ee.Initialize()

In [2]:
#Import relevant packages and functions

%matplotlib notebook

from __future__ import print_function # For py 2.7 compat


import datetime
import ipywidgets
from IPython.display import display
from IPython.display import Image
import traitlets
from IPython.core.display import Javascript

In [3]:
import IPython
IPython.__version__

'5.1.0'

In [4]:
#Import the Digital Elevation Model (DEM)
Elevation = ee.Image("USGS/SRTMGL1_003")
print(Elevation.getInfo())

{u'bands': [{u'crs': u'EPSG:4326', u'crs_transform': [0.0002777777777777778, 0.0, -180.0001388888889, 0.0, -0.0002777777777777778, 60.00013888888889], u'id': u'elevation', u'data_type': {u'max': 32767, u'type': u'PixelType', u'precision': u'int', u'min': -32768}, u'dimensions': [1296001, 417601]}], u'version': 1475598684077000, u'type': u'Image', u'id': u'USGS/SRTMGL1_003', u'properties': {u'provider_url': u'http://www2.jpl.nasa.gov/srtm/', u'system:visualization_0_max': 6000, u'thumb': u'https://mw1.google.com/ges/dd/images/SRTM90_V4_thumb.png', u'system:visualization_0_gamma': 1.6, u'title': u'SRTM Digital Elevation Data 30m', u'description': u"<p>The Shuttle Radar Topography Mission (SRTM, see    <a href='http://onlinelibrary.wiley.com/doi/10.1029/2005RG000183/full'>Farr   et al. 2007</a>) digital elevation data is an international   research effort that obtained digital elevation models on a near-global   scale from 56&deg S to 60&deg N.  This version of SRTM data has a   resolutio

In [5]:
#Local slope - calculates slope in degrees from the DEM
slope = ee.Terrain.slope(Elevation)
print(slope.getInfo())


{u'bands': [{u'crs': u'EPSG:4326', u'crs_transform': [0.0002777777777777778, 0.0, -180.0001388888889, 0.0, -0.0002777777777777778, 60.00013888888889], u'id': u'slope', u'data_type': {u'max': 90.0, u'type': u'PixelType', u'precision': u'float', u'min': 0.0}}], u'type': u'Image'}


In [6]:
# Create a reference to the slope image collection to test that everything worked so far
outImg = ee.ImageCollection(slope).mosaic()
Image(url=outImg.getThumbUrl({'min':0, 'max':180}))
## Define a region roughly covering California -- this is just for now! Update with Senegal later
point = ee.Geometry.Point(-118, 37)
region_california = point.buffer(500000).bounds().getInfo()['coordinates']
# And finally display the image.
Image(url=outImg.getThumbUrl({'region':region_california}))

In [7]:
#Curvature - This calculates the curvature, which is defined as the slope of the slope. 
#You can think of it as the shape (convex, concave, flat) of the area
#While slope helps define the rate of of run-off, curvature helps determine the direction of flow. 
curv  = (Elevation.convolve(ee.Kernel.laplacian8())).resample()
print(curv.getInfo())


{u'bands': [{u'crs': u'EPSG:4326', u'crs_transform': [0.0002777777777777778, 0.0, -180.0001388888889, 0.0, -0.0002777777777777778, 60.00013888888889], u'id': u'elevation', u'data_type': {u'type': u'PixelType', u'precision': u'double'}, u'dimensions': [1296001, 417601]}], u'version': 1475598684077000, u'type': u'Image', u'id': u'USGS/SRTMGL1_003', u'properties': {u'provider_url': u'http://www2.jpl.nasa.gov/srtm/', u'system:visualization_0_max': 6000, u'thumb': u'https://mw1.google.com/ges/dd/images/SRTM90_V4_thumb.png', u'system:visualization_0_gamma': 1.6, u'title': u'SRTM Digital Elevation Data 30m', u'description': u"<p>The Shuttle Radar Topography Mission (SRTM, see    <a href='http://onlinelibrary.wiley.com/doi/10.1029/2005RG000183/full'>Farr   et al. 2007</a>) digital elevation data is an international   research effort that obtained digital elevation models on a near-global   scale from 56&deg S to 60&deg N.  This version of SRTM data has a   resolution of 1 arc-second (approxima

In [8]:
# Create a reference to the curvature image collection to test that it worked
outImg2 = ee.ImageCollection(curv).mosaic()
Image(url=outImg2.getThumbUrl({'min':0, 'max':180}))
## Define a region roughly covering California -- this is just for now! Update with Senegal later
point = ee.Geometry.Point(-118, 37)
region_california = point.buffer(500000).bounds().getInfo()['coordinates']
# And finally display the image.
Image(url=outImg2.getThumbUrl({'region':region_california}))

In [9]:
###Section 2.2 Climate & Precipitation Indicators

#Total precipitation during the flood event from Aug 28th-Sep 4th
    #filters  for precipitation during the period of interest 
#create an image collection for all the days of the year
precip = ee.Image(ee.ImageCollection('NOAA/PERSIANN-CDR').filterDate
    ('2012-08-28', '2012-09-04').select("precipitation").sum())
print(precip.getInfo())

{u'bands': [{u'crs': u'EPSG:4326', u'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0], u'id': u'precipitation', u'data_type': {u'type': u'PixelType', u'precision': u'double'}}], u'type': u'Image'}


In [10]:
## Define event precipitation intensity 
  ##  PINT (mm/day) = P/ndays where P is daily precipitation (mm) and ndays = number of days of storm (day)
    ## this number just tells you how hard it rained each day
pint = precip.divide(8)
print(pint.getInfo())

{u'bands': [{u'crs': u'EPSG:4326', u'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0], u'id': u'precipitation', u'data_type': {u'type': u'PixelType', u'precision': u'double'}}], u'type': u'Image'}


In [11]:
## Annual Storm Intensity Index -- create an image collection for all the days of the year
collection = (ee.ImageCollection('NOAA/PERSIANN-CDR').filterDate('2012-01-01', '2012-12-31'))
# Use the mosaic reducer, to select the most recent pixel in areas of overlap
precip_image = collection.mosaic()
# Define a region roughly covering California
point = ee.Geometry.Point(-118, 37)
region_california = point.buffer(500000).bounds().getInfo()['coordinates']
# And finally display the image.
Image(url=precip_image.getThumbUrl({'region':region_california}))

In [26]:
#Conditional statement to return the image if the precipitation > 1 mm.  
#Otherwise it returns a zero image.
def conditional(image):
        threshold = ee.Image.constant(0.2) ##set threshold with rainy 
        image = image.select('precipitation').gt(threshold)
        return image

In [31]:
## This tells you on days when it rains on average how much does it rain.
## For systems with really intense storms, this will be high. For 
## systems where it rains a lot but just little bits every day, it will be low
## Is this summing the precipitation value or just hte number of days? 
## Either way this code is wrong!!!! Double check this
rainydays =(collection.select('precipitation').map(conditional)).sum()


## Attempting to view the output to see if it works -- big gray square -- not sure
outImg3 = ee.ImageCollection(rainydays).mosaic()
## What should the min and max be here?
Image(url=outImg3.getThumbUrl({'min':0, 'max':10}))
## Define a region roughly covering California -- this is just for now! Update with Senegal later
point = ee.Geometry.Point(-118, 37)
region_california = point.buffer(500000).bounds().getInfo()['coordinates']
# And finally display the image.
Image(url=outImg3.getThumbUrl({'region':region_california}))

In [21]:
#This gives you a sense of how much harder it rained during the storm than it does on any normal rainy day
rainScore = pint.divide(rainydays)

In [32]:
#### Section 2.3 Development Indicators
 
# Impervious surface  -bring in land cover/land use data
cover = ee.Image('ESA/GLOBCOVER_L4_200901_200912_V2_3').select(0)
imp = cover.eq(190)

In [33]:
#### Section 2.4 Hydrologic Indicators 

## Euclidean distance from water features 
## select waterfeatures by filtering for LC labelled 180 (GlobCover category for H20)

## In other code the line directly below was in comments. Is it not supposed to be in here?
waterf= ee.Image("MODIS/MOD44W/MOD44W_005_2000_02_24").select("water_mask");
cover = ee.Image('ESA/GLOBCOVER_L4_200901_200912_V2_3').select(0);
waterf = cover.eq(210)
imp = cover.eq(190)
print(cover.getInfo())

{u'bands': [{u'crs': u'EPSG:4326', u'crs_transform': [0.002777777777777778, 0.0, -180.0013888888889, 0.0, -0.002777777777777778, 90.00138888888888], u'dimensions': [129600, 55800], u'data_type': {u'max': 255, u'type': u'PixelType', u'precision': u'int', u'min': 0}, u'id': u'landcover', u'properties': {u'system:nodata_value': 0.0, u'system:interpolate': 0}}], u'version': 1476468451314000, u'type': u'Image', u'id': u'ESA/GLOBCOVER_L4_200901_200912_V2_3', u'properties': {u'system:time_end': 1395316593836, u'system:visualization_0_bands': u'landcover', u'provider_url': u'http://dup.esrin.esa.int/page_globcover.php', u'system:asset_size': 424749697, u'title': u'GlobCover: Global Land Cover Map', u'landcover_class_values': [11, 14, 20, 30, 40, 50, 60, 70, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230], u'landcover_class_names': [u'Post-flooding or irrigated croplands', u'Rainfed croplands', u'Mosaic Cropland (50-70%) / Vegetation (grassland, shrubland, forest) (20-

In [34]:
# Create a distance kernel -defining area of analysis
kern = ee.Kernel.euclidean(2000, 'meters')

In [35]:
# associate the viewing distance (kern) with the water features
distance = waterf.distance(kern)

In [36]:
# calculate distance from rivers using the NHD data and the distance variable -- hopefully will work once i enter
# in Senegal specific info
rdist = nhd.distance().clip(fc)

NameError: name 'nhd' is not defined

In [37]:
##Topographic Wetness Index (TWI)
  #Preprocessing includes generating flow accumulation, catchment area & slope --15 sec flow accumulation
FLOWAC = ee.Image('WWF/HydroSHEDS/15ACC')

In [44]:
## Test image - image test not working -- maybe be image, may be symbology
Image(url=FLOWAC.getThumbUrl({'min':0, 'max':30}))
## Define a region roughly covering California -- this is just for now! Update with Senegal later
point = ee.Geometry.Point(-118, 37)
region_california = point.buffer(100).bounds().getInfo()['coordinates']
# And finally display the image.
Image(url=FLOWAC.getThumbUrl({'region':region_california}))

In [45]:
##flow accumulation in the flood zone - test this image once I have Senegal data
floodflow = FLOWAC.clip(fc)

NameError: name 'fc' is not defined

In [46]:
## create layer of just 1's 
imageof1 = ee.Image.constant(1)

In [47]:
## add +1 to flow accumulation -this is so you don't get zeros when you take the log -- Floodflow not defined yet
floodflowplus1 = floodflow.add(imageof1)

NameError: name 'floodflow' is not defined

In [48]:
## pixel dimensions of flow accumulation
pixel_area = ee.Image.pixelArea().mask(floodflow.mask())

NameError: name 'floodflow' is not defined

In [49]:
## now calculate the upstream area for each pixel using 
  # contributing catchment area according to topmodel topographic index equation.
      ## CA= (flow accum + 1 * cell^2)
        ## calculated by multiplying the number of pixels 
        ## flowing into a given pixel * the area 
CA = floodflowplus1.multiply(pixel_area)

NameError: name 'floodflowplus1' is not defined

In [50]:
## add a very low number to the slope so that zero 
  ## slopes dont get counted out of the index
imageofalmost0 = ee.Image.constant(0.0000001)

In [51]:
## getting new slope with no zero
nonzeroslopes = slope.add(imageofalmost0)

In [53]:
## convert to radians - originally was divided by 280 - should be 180?
sloperadians = nonzeroslopes.multiply(3.141593/180)

In [57]:
## calcualte the tan of the slope for TI equation
tanslope = sloperadians.tan()
clipped2 = tanslope

In [58]:
##topographic index is LN([flowaccum+1*cellarea]/tanslope)
      # It is defined as ln(a/tanβ) where a is the local
      # upslope area draining through a certain point per unit contour
    # length and tanβ is the local slope.
      # note that both TWI and SPI assume steady state
topoindex = CA.divide(clipped2)
twi = topoindex.log()

NameError: name 'CA' is not defined

In [59]:
## Stream Power Index (SPI)
  ## is just CA/slope (Gallant 2000 Terrain Analysis)
streampower = CA.multiply(clipped2) #final SPI
spi = streampower.clip(fc) #clip to watershed

NameError: name 'CA' is not defined

In [61]:
# Aspect (snowmelt )
## tells you the direction of the slope (0-360 degrees) -important for snowmelt
aspect = ee.Terrain.aspect(Elevation)

In [63]:
## HEIGHT ABOVE NEAREST DRAINAGE - how do i get this global hand data
hand = ee.ImageCollection('users/gena/global-hand/hand-100').mosaic()

In [65]:
def radians(img):
    return img.toFloat().multiply(3.1415927).divide(180)

def hillshade(az, ze, slope, aspect):
    azimuth = radians(ee.Image(az))
    zenith = radians(ee.Image(ze))
    return azimuth.subtract(aspect).cos().multiply(slope.sin()).multiply(zenith.sin()).add(zenith.cos().multiply(slope.cos()))

def hillshadeit(image, elevation, weight, height_multiplier, azimuth, zenith):
    hsv  = image.unitScale(0, 255).rgbtohsv()
    
    terrain = ee.call('Terrain', elevation.multiply(height_multiplier))
    slope = radians(terrain.select(['slope']))

    aspect = radians(terrain.select(['aspect'])).resample('bicubic')
    hs = hillshade(azimuth, zenith, slope, aspect).resample('bicubic')

    intensity = hs.multiply(weight).multiply(hsv.select('value'));
    huesat = hsv.select('hue', 'saturation');

    return ee.Image.cat(huesat, intensity).hsvtorgb()

In [66]:
## global DEM mosaic - I need to figure out how to mosaic these correctly. This is where I've gotten to. 11/30/16
dem =
  ee.Image('users/gena/ViewfinderpanoramaDEM/VFP_DEM1'),
  ee.Image('users/gena/ViewfinderpanoramaDEM/VFP_DEM2'),
  ee.Image('users/gena/ViewfinderpanoramaDEM/VFP_DEM3'),
  ee.Image('users/gena/ViewfinderpanoramaDEM/VFP_DEM4'),
  ee.Image('users/gena/ViewfinderpanoramaDEM/VFP_DEM5'),
  ee.Image('users/gena/ViewfinderpanoramaDEM/VFP_DEM6'),
  ee.Image('users/gena/ViewfinderpanoramaDEM/VFP_DEM7'),
  ee.Image('users/gena/ViewfinderpanoramaDEM/VFP_DEM8'),
  elevation

SyntaxError: invalid syntax (<ipython-input-66-06305e57a9eb>, line 1)

In [94]:
dem = ee.ImageCollection(dem).map(function(i) { return i.rename('elevation').add(0)}

SyntaxError: invalid syntax (<ipython-input-94-b97a37ed2c7c>, line 1)

In [95]:
var handVis = function(hsWeight, heightMultiplier, azimuth, zenith) { return dem.map(function(i) { 
  var handRGB = hand.visualize({min:-1, max:50, palette:colors_hand});
  return hillshadeit(handRGB, i, hsWeight, heightMultiplier, azimuth, zenith); 
}).mosaic()};

SyntaxError: invalid syntax (<ipython-input-95-3ddc9a413299>, line 1)

In [97]:
azimuth = 90
zenith = 20
hsWeight = 1.1
heightMultiplier = 4

In [98]:
dfo3180 = ee.Image('users/jonathanasullivan/geeOtsuDFOAssets_Senegal/Senegal_Otsu_DFO3180_3Day')

In [99]:
mod44_water = mod44.select([0]).clip(fc)
hansen_w = hansen.select('datamask').eq(2)
hansen_water = hansen_w.clip(fc)

NameError: name 'mod44' is not defined

In [100]:
hansen = hansen.select('datamask')

NameError: name 'hansen' is not defined

In [101]:
hansenLand = hansen.select([0]).remap([0,1,2], [1,1,0])

NameError: name 'hansen' is not defined

In [102]:
region =fc
variables = (elevation
  .addBands(slope)
  .addBands(curv)
  .addBands(pint)
  .addBands(imp)
  .addBands(precip)
  .addBands(twi)
  .addBands(spi)
  .addBands(rdist)
  .addBands(aspect)
  .addBands(hand));

NameError: name 'fc' is not defined

In [103]:
variables = variables.mask(hansen)

NameError: name 'variables' is not defined

In [104]:
bands = ['elevation','slope', 'elevation_1','precipitation', 'landcover', 'precipitation_1', 
        'b1', 'b1_1', 'distance', 'aspect', 'b1_2']

In [105]:
only_flooded_points= dfo3180.mask(dfo3180.select(['flooded'],['flood']).eq(1))

In [106]:
not_flooded_points= dfo3180.mask(dfo3180.select(['flooded'],['flood']).neq(1))

In [111]:
only_400_flood_points = only_flooded_points.addBands(ee.Image.pixelLonLat()).sample({
    'region': fc.geometry().convexHull(),
    'scale': 250,
    'seed': 0,
    'numPixels': 300000,
  }).randomColumn().sort('random').limit(4000).map(function(f) { 
    return ee.Feature(
      ee.Geometry.Point([f.get('longitude'), f.get('latitude')]), f.toDictionary())
  })

SyntaxError: invalid syntax (<ipython-input-111-d2feb51e54d4>, line 6)

In [113]:
only_400_not_flooded_points = not_flooded_points.addBands(ee.Image.pixelLonLat()).sample({
    'region': fc.geometry().convexHull(),
    'scale': 250,
    'seed': 0,
    'numPixels': 300000,
    'dropNulls': true
  }).randomColumn().sort('random').limit(4000).map(function(f) { 
    return ee.Feature(
      ee.Geometry.Point([f.get('longitude'), f.get('latitude')]), f.toDictionary())
  })

SyntaxError: invalid syntax (<ipython-input-113-c41cd32090a5>, line 3)

In [114]:
allPoints = only_400_flood_points.merge(only_400_not_flooded_points)

NameError: name 'only_400_flood_points' is not defined

In [115]:
trainingTesting = allPoints.randomColumn()

NameError: name 'allPoints' is not defined

In [117]:
setOneTrain = trainingTesting.filter(ee.Filter.rangeContains('random', .1, 1))
setOneTest = trainingTesting.filter(ee.Filter.rangeContains('random', 0, 0.099))

setTwoBefore= trainingTesting.filter(ee.Filter.rangeContains('random', 0, 0.099))
setTwoAfter= trainingTesting.filter((ee.Filter.rangeContains('random', .2, 1)))
setTwoTrain = setTwoBefore.merge(setTwoAfter)
setTwoTest = trainingTesting.filter(ee.Filter.rangeContains('random', 0.1, .199))

setThreeBefore= trainingTesting.filter(ee.Filter.rangeContains('random', 0, .199))
setThreeAfter= trainingTesting.filter((ee.Filter.rangeContains('random', .3, 1)))
setThreeTrain = setThreeBefore.merge(setThreeAfter)
setThreeTest = trainingTesting.filter(ee.Filter.rangeContains('random', .2, .299))

setFourBefore= trainingTesting.filter(ee.Filter.rangeContains('random', 0, .299))
setFourAfter= trainingTesting.filter((ee.Filter.rangeContains('random', .4, 1)))
setFourTrain = setFourBefore.merge(setFourAfter)
setFourTest = trainingTesting.filter(ee.Filter.rangeContains('random', .3, .399))

setFiveBefore= trainingTesting.filter(ee.Filter.rangeContains('random', 0, .399))
setFiveAfter= trainingTesting.filter((ee.Filter.rangeContains('random', .5, 1)))
setFiveTrain = setFiveBefore.merge(setFiveAfter)
setFiveTest = trainingTesting.filter(ee.Filter.rangeContains('random', .4, .499))

setSixBefore= trainingTesting.filter(ee.Filter.rangeContains('random', 0, .499))
setSixAfter= trainingTesting.filter((ee.Filter.rangeContains('random', .6, 1)))
setSixTrain = setSixBefore.merge(setSixAfter)
setSixTest = trainingTesting.filter(ee.Filter.rangeContains('random', .5, .599))

setSevenBefore= trainingTesting.filter(ee.Filter.rangeContains('random', 0, .599))
setSevenAfter= trainingTesting.filter((ee.Filter.rangeContains('random', .7, 1)))
setSevenTrain = setSevenBefore.merge(setSevenAfter)
setSevenTest = trainingTesting.filter(ee.Filter.rangeContains('random', .6, .699))

setEightBefore= trainingTesting.filter(ee.Filter.rangeContains('random', 0, .699))
setEightAfter= trainingTesting.filter((ee.Filter.rangeContains('random', .8, 1)))
setEightTrain = setEightBefore.merge(setEightAfter)
setEightTest = trainingTesting.filter(ee.Filter.rangeContains('random', .7, .799))
  
setNineBefore= trainingTesting.filter(ee.Filter.rangeContains('random', 0, .799))
setNineAfter= trainingTesting.filter((ee.Filter.rangeContains('random', .9, 1)))
setNineTrain = setNineBefore.merge(setNineAfter)
setNineTest = trainingTesting.filter(ee.Filter.rangeContains('random', .8, .899))

setTenTrain = trainingTesting.filter(ee.Filter.rangeContains('random', 0, .899))
setTenTest = trainingTesting.filter(ee.Filter.rangeContains('random', .9, .999))


NameError: name 'trainingTesting' is not defined

In [118]:
testOneTrain = variables.sampleRegions(setOneTrain, ['flooded'], 250)
testOneTest = variables.sampleRegions(setOneTest, ['flooded'], 250)

testTwoTrain = variables.sampleRegions(setTwoTrain, ['flooded'], 250)
testTwoTest = variables.sampleRegions(setTwoTest, ['flooded'], 250)

testThreeTrain = variables.sampleRegions(setThreeTrain, ['flooded'], 250)
testThreeTest = variables.sampleRegions(setThreeTest, ['flooded'], 250)

testFourTrain = variables.sampleRegions(setFourTrain, ['flooded'], 250)
testFourTest = variables.sampleRegions(setFourTest, ['flooded'], 250)

testFiveTrain = variables.sampleRegions(setFiveTrain, ['flooded'], 250)
testFiveTest = variables.sampleRegions(setFiveTest, ['flooded'], 250)

testSixTrain = variables.sampleRegions(setSixTrain, ['flooded'], 250)
testSixTest = variables.sampleRegions(setSixTest, ['flooded'], 250)

testSevenTrain = variables.sampleRegions(setSevenTrain, ['flooded'], 250)
testSevenTest = variables.sampleRegions(setSevenTest, ['flooded'], 250)

testEightTrain = variables.sampleRegions(setEightTrain, ['flooded'], 250)
testEightTest = variables.sampleRegions(setEightTest, ['flooded'], 250)

testNineTrain = variables.sampleRegions(setNineTrain, ['flooded'], 250)
testNineTest = variables.sampleRegions(setNineTest, ['flooded'], 250)

testTenTrain = variables.sampleRegions(setTenTrain, ['flooded'], 250)
testTenTest = variables.sampleRegions(setTenTest, ['flooded'], 250)


NameError: name 'variables' is not defined